In [6]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes

from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding

from cryptography.hazmat.primitives.ciphers.aead import AESGCM as aes

from cryptography.hazmat.backends import default_backend

from cryptography.exceptions import InvalidSignature, InvalidTag

from os import urandom

## [RSA](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/rsa/)

### Keys generation

In [2]:
def GenRSAKeyPair(key_size=512):
    private_key = rsa.generate_private_key(public_exponent=65537,
                                           key_size=key_size,
                                           backend=default_backend())
    return private_key

### Keys Uploading and Downloading

In [3]:
def WriteRSAKeys(private_key, password,
                 private_key_filename="private_key",
                 public_key_filename="public_key"):
    bytes_password = bytes(password, "utf-8")
    private_key_bytes = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(bytes_password))
    if not private_key_filename.endswith(".pem"):
        private_key_filename = private_key_filename + ".pem"
    with open(private_key_filename, "wb") as file:
        file.write(private_key_bytes)

    public_key = private_key.public_key()
    public_key_bytes = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo)
    if not public_key_filename.endswith(".pem"):
        public_key_filename = public_key_filename + ".pem"
    with open(public_key_filename, "wb") as file:
        file.write(public_key_bytes)
    return

def ReadRSAPrivateKey(password,
                      private_key_filename="private_key"):
    bytes_password = bytes(password, "utf-8")
    if not private_key_filename.endswith(".pem"):
        private_key_filename = private_key_filename + ".pem"
    with open(private_key_filename, "rb") as file:
        private_key = serialization.load_pem_private_key(
            data=file.read(),
            password=bytes_password,
            backend=default_backend())
    return private_key

def ReadRSAPublicKey(public_key_filename="public_key"):
    if not public_key_filename.endswith(".pem"):
        public_key_filename = public_key_filename + ".pem"
    with open(public_key_filename, "rb") as file:
        public_key = serialization.load_pem_public_key(
            data=file.read(),
            backend=default_backend())
    return public_key

### Signing and Verification

In [4]:
def SignUsingRSA(message, private_key):
    signature = private_key.sign(
        data=message,
        padding=padding.PSS(
            mgf=padding.MGF1(hashes.SHA512()),
            salt_length=padding.PSS.MAX_LENGTH),
        algorithm=hashes.SHA512())
    return signature

def VerifyUsingRSA(signature, message, public_key):
    try:
        public_key.verify(
            signature=signature,
            data=message,
            padding=padding.PSS(
                mgf=padding.MGF1(hashes.SHA512()),
                salt_length=padding.PSS.MAX_LENGTH),
            algorithm=hashes.SHA512())
        return True
    except InvalidSignature:
        return False

### Encryption and Decryption

In [5]:
def EncryptUsingRSA(message, public_key):
    cipher = public_key.encrypt(
        plaintext=message,
        padding=padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA512()),
            algorithm=hashes.SHA512(),
            label=None))
    return cipher

def DecryptUsingRSA(cipher, private_key):
    message = private_key.decrypt(
        ciphertext=cipher,
        padding=padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA512()),
            algorithm=hashes.SHA512(),
            label=None))
    return message

## [AES](https://cryptography.io/en/latest/hazmat/primitives/aead/#)

### Key generation, Encryption, Signing and Decryption, Verification

In [7]:
def GenerateAESKeyAndNonce(key_size=256, nonce_size=96):
    key = aes.generate_key(bit_length=key_size)
    nonce = urandom(nonce_size)
    return key, nonce

def EncryptAndSignUsingAES(message, key, nonce):
    aes_obj = aes(key)
    cipher = aes_obj.encrypt(nonce, message, None)
    return cipher

def DecryptAndVerifyUsingAES(cipher, key, nonce):
    aes_obj = aes(key)
    try:
        message = aes_obj.decrypt(nonce, cipher, None)
        verification = True
    except InvalidTag:
        message = b""
        verification = False
    return verification, message

## Interface

In [10]:
def SignAndEncryptMessage(message, sender_private_key, receiver_public_key):
    pass

def DecryptAndVerifyMessage(prefix_cipher_signature, sender_public_key, receiver_private_key):
    pass